In [16]:
from qiskit.quantum_info import Pauli
from gibbs.learning.klocal_pauli_basis import KLocalPauliBasis
import numpy as np
from itertools import product
import plotly.express as px
from gibbs.dataclass import GibbsResult,get_results
import numpy as np
from gibbs.utils import simple_purify_hamiltonian,printarray,spectral_dec,candidate, lattice_hamiltonian, noise_hamiltonian
import matplotlib.pyplot as plt
from gibbs.learning.bayesian_learning_v2 import BayesianLearning
from gibbs.learning.constraint_matrix import ConstraintMatrixFactory
from gibbs.plots.plots import compare_preparations, axis_locality,preparation
import plotly.express as px
from qiskit.quantum_info import state_fidelity,SparsePauliOp


%load_ext autoreload
%autoreload 2
result = get_results("../scripts_lap/job_13_qasm")[0]
preparation(result)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(111,) (111,) (111,)


In [17]:
from qiskit.primitives import BackendEstimator
from qiskit.providers.fake_provider import FakeAuckland
estimator = BackendEstimator(backend = FakeAuckland())
cmat = ConstraintMatrixFactory(result.num_qubits,2,3,estimator=estimator)
K,E = cmat.create_cmat(result.state_vector(-1),shots=1e5)
E_real = np.sign(K)[:,:,None]*E
randomness = np.random.normal(0,1,E_real.shape[-1])
Noise = np.dot(E_real,randomness)

In [34]:
ss = []
vv = []
shots_list = np.logspace(4,8,100)
for shots in shots_list:
    eps = shots**-0.5 
    s,v = spectral_dec(K + eps*Noise)
    ss.append(s)
    vv.append(v)
error = np.array([min(np.linalg.norm(vvv[-1]-sign*vv[-1][-1]) for sign in (+1,-1)) for vvv in vv])
px.scatter(x=shots_list,y=error**(1),log_x=True,log_y=True)